Need `mamba install -c conda-forge hvplot geoviews nodejs bokeh holoviews panel`.

In [ ]:
import pypsa
import atlite

import pandas as pd
import numpy as np
import geopandas as gpd
import xarray as xr
import networkx as nx

import panel as pn
import panel.widgets as pnw
import holoviews as hv

import cartopy.crs as ccrs

import hvplot.pandas
import hvplot.xarray
import hvplot.networkx as hvnx

In [ ]:
from bokeh.models.formatters import DatetimeTickFormatter
pn.extension()

In [ ]:
import yaml
with open("config.yaml") as file:
    config = yaml.safe_load(file)

colors = config["plotting"]["tech_colors"]

## Solved Network

In [ ]:
n = pypsa.Network(
    "../pr/pypsa-eur-sec/results/your-run-name-overnight-dev/postnetworks/elec_s_60_lv1.25__Co2L0p0-365H-T-H-B-I-solar+p3-dist1_2030.nc"
)

## Geometry Polygon Data

In [ ]:
# shapes
nodes = gpd.read_file("../pr/pypsa-eur/resources/regions_onshore_elec_s_60.geojson").set_index('name')
cts = gpd.read_file("../pr/pypsa-eur/resources/country_shapes.geojson").set_index('name')

## Model Inputs

In [ ]:
# country-level data
co2 = pd.read_csv("../pr/pypsa-eur-sec/resources/co2_totals.csv", index_col=0)
energy = pd.read_csv("../pr/pypsa-eur-sec/resources/energy_totals.csv", index_col=0)
transport = pd.read_csv("../pr/pypsa-eur-sec/resources/transport_data.csv", index_col=0)
biomass = pd.read_csv("../pr/pypsa-eur-sec/resources/biomass_potentials.csv", index_col=0)

In [ ]:
# nodal-level data
pop = pd.read_csv("../pr/pypsa-eur-sec/resources/pop_layout_elec_s_60.csv", index_col=0)
idist = pd.read_csv("../pr/pypsa-eur-sec/resources/industrial_distribution_key_elec_s_60.csv", index_col=0)
ienergy = pd.read_csv("../pr/pypsa-eur-sec/resources/industrial_energy_demand_elec_s_60.csv", index_col=0)
iproduction = pd.read_csv("../pr/pypsa-eur-sec/resources/industrial_production_elec_s_60.csv", index_col=0)
ienergy["total"] = ienergy.sum(axis=1)
iproduction["total"] = iproduction.sum(axis=1)

In [1043]:
def plot_geo(gdf, df, options, clim=None):

    selector = pnw.Select(options=options)

    def _plot(select):
        return gdf.hvplot(
            geo=True,
            frame_height=600,
            c=df[select],
            #tiles='CartoLight',
            alpha=0.7,
            line_width=0.8,
            cmap='Greens',
            clim=clim
        ).opts(
            active_tools=['pan', 'wheel_zoom']
        )

    plot = pn.bind(_plot, selector)
    widgets  = pn.Column(selector, plot)
    return widgets

In [ ]:
plot_geo(cts, biomass, list(biomass.columns))

In [ ]:
plot_geo(cts, transport, list(transport.columns))

In [ ]:
plot_geo(nodes, idist, list(idist.columns))

In [ ]:
plot_geo(nodes, iproduction, list(iproduction.columns))

In [ ]:
plot_geo(nodes, pop, ["total", "urban", 'rural'])

In [ ]:
plot_geo(cts, co2, list(co2.columns))

### Renewable Potentials

In [ ]:
cfs = n.generators_t.p_max_pu.groupby([n.generators.carrier, n.generators.bus.map(n.buses.location)], axis=1).sum().sum().unstack(0)

In [ ]:
plot_geo(nodes, cfs, list(cfs.columns))

In [ ]:
pot = n.generators.p_nom_max.groupby([n.generators.carrier, n.generators.bus.map(n.buses.location)]).sum().unstack(0)

In [ ]:
pot.drop(index="EU", columns=['gas', 'oil', 'ror'], inplace=True)

In [ ]:
plot_geo(nodes, pot, list(pot.columns))

### Nodal Capacities and Costs

In [1015]:
term_p = "p_nom_opt"
term_e = "e_nom_opt"

In [1031]:
term_e = "capital_cost * (e_nom_opt-e_nom)"
term_p = "capital_cost * (p_nom_opt-p_nom)"

In [1032]:
gen = n.generators.eval(term_p).groupby([n.generators.carrier, n.generators.bus.map(n.buses.location)]).sum()
sto = n.stores.eval(term_e).groupby([n.stores.carrier, n.stores.bus.map(n.buses.location)]).sum()
local_links = n.links.loc[n.links.bus0.map(n.buses.location) == n.links.bus1.map(n.buses.location)]
link = local_links.eval(term_p).groupby([local_links.carrier, local_links.bus0.map(n.buses.location)]).sum()
su = n.storage_units.eval(term_p).groupby([n.storage_units.carrier, n.storage_units.bus]).sum()

In [1037]:
gen = gen.unstack().drop("EU", axis=1).dropna(how='all')
link = link.unstack().drop("EU", axis=1).dropna(how='all')
sto = sto.unstack().drop("EU", axis=1).dropna(how='all')
su = su.unstack()

In [1039]:
cap = pd.concat([gen, sto, link, su]).T.div(1e9)

In [1046]:
plot_geo(nodes, cap, list(cap.columns))#, clim=(0,25))

Column
    [0] Select(options=['offwind-ac', ...], value='offwind-ac')
    [1] ParamFunction(function)

## Networks

In [ ]:
G_lines = nx.from_pandas_edgelist(n.lines, 'bus0', 'bus1', edge_attr='s_nom_opt')

In [ ]:
G_links = nx.from_pandas_edgelist(n.links.loc[n.links.carrier=='DC'], 'bus0', 'bus1', edge_attr='p_nom_opt')

In [ ]:
H2 = n.links.loc[n.links.carrier=='H2 pipeline']
H2["location0"] = H2.bus0.apply(lambda x: x[:-3])
H2["location1"] = H2.bus1.apply(lambda x: x[:-3])
G_H2 = nx.from_pandas_edgelist(H2, 'location0', 'location1', edge_attr='p_nom_opt')
electrolysis = n.links.loc[n.links.carrier=='H2 Electrolysis'].groupby("bus0").p_nom_opt.sum()
nx.set_node_attributes(G_H2, electrolysis, "electrolysis")

In [ ]:
pos = n.buses.loc[n.buses.carrier=='AC', ["x", "y"]].apply(tuple, axis=1).to_dict()

In [ ]:
nodes.hvplot(
    geo=True,
    color='whitesmoke',
    line_color='grey',
    line_width=0.5,
    #transform=ccrs.EuroPP(),
) * \
hvnx.draw(
    G_links, 
    pos=pos,
    width=1000,
    height=800,
    node_size=0,
    edge_color='navy',
    edge_width=hv.dim('p_nom_opt') / 3e3,
    geo=True,
    #crs=ccrs.EuroPP(),
    inspection_policy='edges'
) * \
hvnx.draw(
    G_lines, 
    pos=pos,
    width=1000,
    height=800,
    node_size=40,
    node_color='gray',
    edge_color='firebrick',
    edge_width=hv.dim('s_nom_opt') / 3e3,
    geo=True,
    #crs=ccrs.EuroPP(),
    inspection_policy='edges'
).opts(
    active_tools=['pan', 'wheel_zoom']
)

In [ ]:
nodes.hvplot(
    geo=True,
    color='whitesmoke',
    line_color='grey',
    line_width=0.5,
    #transform=ccrs.EuroPP(),
) * \
hvnx.draw(
    G_H2, 
    pos=pos,
    width=1000,
    height=800,
    edge_color='cyan',
    edge_width=hv.dim('p_nom_opt') / 3e3,
    node_color='magenta',
    node_size=hv.dim("electrolysis") / 2e2,
    geo=True,
    #crs=ccrs.EuroPP(),
    inspection_policy='edges'
).opts(
    active_tools=['pan', 'wheel_zoom']
)

## System time series

In [ ]:
# one resampled version, one hourly version

In [ ]:
load = n.loads_t.p_set.groupby(n.loads.carrier, axis=1).sum()
formatter = DatetimeTickFormatter(months='%b')

In [ ]:
load.hvplot.area(width=1000, stacked=True, xformatter=formatter)

In [ ]:
selection = ["offwind-ac", "offwind-dc", "onwind", "ror", "solar"]
cfs = n.generators_t.p_max_pu.groupby(n.generators.carrier, axis=1).mean()[selection]

In [ ]:
cfs.hvplot.line(width=1000, xformatter=formatter)

In [1089]:
gen = n.generators_t.p.groupby(n.generators.carrier, axis=1).sum()

In [1090]:
su = n.storage_units_t.p.groupby(n.storage_units.carrier, axis=1).sum()

In [1091]:
df = pd.concat([gen, su], axis=1)

In [1092]:
df.hvplot.area(width=1000, height=500, line_width=0, title="electricity generation")

:NdOverlay   [carrier]
   :Area   [name]   (value,Baseline)

# Cutouts

In [ ]:
era5 = atlite.Cutout("../pr/pypsa-eur/cutouts/europe-2013-era5.nc")
sarah = atlite.Cutout("../pr/pypsa-eur/cutouts/europe-2013-sarah.nc")

In [ ]:
sarah.data.hvplot.quadmesh(
    'x', 'y', 'influx_direct',
    #projection=ccrs.EuroPP(),
    frame_height=700,
    cmap='Blues',
    coastline=True,
    project=True,
    geo=True,
    rasterize=True,
    ylim=(34,72),
    xlim=(-12,34),
    clim=(0,1200),
    #tiles='CartoLight'
    #datashade=True,
)

In [ ]:
era5.data.hvplot.contourf(
    'x', 'y', 'temperature',
    #transform=ccrs.EuroPP(),
    frame_height=700,
    cmap='Reds',
    #features=["borders"],
    coastline=True,
    levels=15,
    project=True,
    geo=True,
    ylim=(34,72),
    xlim=(-12,34),
    #alpha=0.8,
    #tiles='CartoLight
)

In [ ]:
import sys, os
sys.path.insert(0, os.getcwd() + "/scripts")
from scripts.plot_summary import rename_techs, preferred_order

## Outputs

In [ ]:
cost_df = pd.read_csv(
    "../pr/pypsa-eur-sec/results/your-run-name-overnight-dev/csvs/costs.csv",
    index_col=list(range(3)),
    header=list(range(4))
)
df = cost_df.groupby(cost_df.index.get_level_values(2)).sum()
df = df / 1e9
df = df.groupby(df.index.map(rename_techs)).sum()

to_drop = df.index[df.max(axis=1) < 1.]
                   
new_index = preferred_order.intersection(df.index).append(df.index.difference(preferred_order))
new_columns = df.sum().sort_values().index

In [ ]:
df.columns = [', '.join(col).strip() for col in df.columns.values]

In [ ]:
df.T.hvplot.bar(stacked=True, rot=65, width=900, height=1060, color='Category', cmap=colors)

In [ ]:
energy_df = pd.read_csv(
    "../pr/pypsa-eur-sec/results/your-run-name-overnight-dev/csvs/energy.csv",
    index_col=list(range(2)),
    header=list(range(4))
)
df = energy_df.groupby(energy_df.index.get_level_values(1)).sum()
df = df / 1e6
df = df.groupby(df.index.map(rename_techs)).sum()
to_drop = df.index[df.abs().max(axis=1) < 50]
df = df.drop(to_drop)
new_index = preferred_order.intersection(df.index).append(df.index.difference(preferred_order))
new_columns = df.columns.sort_values()

In [ ]:
df.columns = [', '.join(col).strip() for col in df.columns.values]

In [ ]:
df.T.hvplot.bar(stacked=True, rot=65, width=900, height=1060, color='Category', cmap=colors)

## Sankey
as https://holoviews.org/gallery/demos/bokeh/energy_sankey.html

In [ ]:
edges = pd.read_csv('connection.csv')
sankey = hv.Sankey(edges, label='Energy Diagram')
sankey.opts(label_position='left', edge_color='target', node_color='index', cmap=colors)

## Industry Sector Ratios

In [ ]:
iratios = pd.read_csv("../pr/pypsa-eur-sec/resources/industry_sector_ratios.csv", index_col=0)

In [ ]:
iratios.T.hvplot.barh(stacked=True, width=1000, height=400, title="Industry Sector Ratios [MWh/t material]")

## Hotmaps Raw

In [ ]:
def prepare_hotmaps_database(regions):
    """
    Load hotmaps database of industrial sites and map onto bus regions.
    """

    df = pd.read_csv("../pr/pypsa-eur-sec/data/Industrial_Database.csv", sep=";", index_col=0)

    df[["srid", "coordinates"]] = df.geom.str.split(';', expand=True)

    # remove those sites without valid locations
    df.drop(df.index[df.coordinates.isna()], inplace=True)

    df['coordinates'] = gpd.GeoSeries.from_wkt(df['coordinates'])

    gdf = gpd.GeoDataFrame(df, geometry='coordinates', crs="EPSG:4326")

    gdf = gpd.sjoin(gdf, regions, how="inner", op='within')

    gdf.rename(columns={"index_right": "bus"}, inplace=True)
    gdf["country"] = gdf.bus.str[:2]

    return gdf

In [ ]:
hotmaps = prepare_hotmaps_database(nodes)

hotmaps["geometry"] = hotmaps.coordinates
hotmaps["Latitude"] = hotmaps.geometry.y
hotmaps["Longitude"] = hotmaps.geometry.x

plot = hotmaps.hvplot(
    geo=True,
    frame_height=800,
    by='Subsector',
    size=hotmaps["Emissions_ETS_2014"] / 2e3,
    alpha=0.4,
    tiles='CartoLight',
).opts(
    active_tools=['pan', 'wheel_zoom']
)
hvplot.save(plot, 'hotmaps.html')